In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v3.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
89056,Luke_Freeman,36,3,0,Sheffield United,Leicester City,0.299994,1.047611,False,0,...,0,2,0.0,-56,1,57,0,2020-07-16T17:00:00Z,1920,0.00
89057,Kelland_Watts,36,2,0,Newcastle United,Tottenham Hotspur,0.605226,1.748097,True,0,...,3,1,0.0,194,578,384,0,2020-07-15T17:00:00Z,1920,0.00
89058,Shkodran_Mustafi,36,2,0,Arsenal,Liverpool,1.500845,2.458682,True,0,...,1,2,0.0,-164,487,651,0,2020-07-15T19:15:00Z,1920,1.00
89059,Mace_Goodridge,36,3,0,Burnley,Wolverhampton Wanderers,0.361664,0.738141,True,0,...,1,1,0.0,-8,21,29,0,2020-07-15T17:00:00Z,1920,1.00
89060,Dan_Gosling,36,3,69,Bournemouth,Manchester City,0.615026,2.448707,False,2,...,1,2,39.0,-85,56,141,0,2020-07-15T17:00:00Z,1920,1.00
89061,Tammy_Abraham,36,4,4,Chelsea,Norwich,2.070909,0.311665,True,1,...,0,1,0.0,5616,29210,23594,0,2020-07-14T19:15:00Z,1920,1.00
89062,Nicolás_Otamendi,36,2,90,Manchester City,Bournemouth,2.448707,0.615026,True,2,...,1,2,0.0,-1727,6110,7837,0,2020-07-15T17:00:00Z,1920,1.00
89063,James_Daly,36,4,0,Crystal Palace,Manchester United,0.423683,1.645057,True,0,...,2,0,0.0,-10,5,15,0,2020-07-16T19:15:00Z,1920,0.00
89064,Declan_Rice,36,3,90,West Ham United,Watford,0.719313,0.481612,True,10,...,1,3,15.0,-279,2243,2522,0,2020-07-17T19:00:00Z,1920,1.00
89065,Connor_Wickham,36,4,0,Crystal Palace,Manchester United,0.423683,1.645057,True,0,...,2,0,0.0,-481,49,530,0,2020-07-16T19:15:00Z,1920,0.00


In [5]:
# df_train[(df_train['gw']==24) & (df_train['season']=='1920') & (df_train['team']=='Liverpool')]

In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

37

In [7]:
# last_gw = 28

In [8]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [9]:
df_latest.drop_duplicates(inplace=True)

In [10]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,37,2,0,Bournemouth,Southampton,0.613218,0.486089,True,0,...,45425,2,0,0.0,-144,4,148,0,2020-07-19T13:00:00Z,1920
1,Paulo_Gazzaniga,37,1,0,Tottenham Hotspur,Leicester City,1.737985,1.044531,True,0,...,114835,0,3,0.0,-1367,758,2125,0,2020-07-19T15:00:00Z,1920
2,John_Egan,37,2,90,Sheffield United,Everton,0.300107,1.032597,True,2,...,551019,1,0,20.0,67362,76679,9317,0,2020-07-20T17:00:00Z,1920
3,Max_Meyer,37,3,0,Crystal Palace,Wolverhampton Wanderers,0.429399,0.735971,False,0,...,5971,0,2,0.0,-9,4,13,0,2020-07-20T19:15:00Z,1920
4,Fabian_Delph,37,2,0,Everton,Sheffield United,1.032597,0.300107,False,0,...,12786,1,0,0.0,-23,18,41,0,2020-07-20T17:00:00Z,1920
5,James_Justin,37,2,90,Leicester City,Tottenham Hotspur,1.044531,1.737985,False,-1,...,49282,0,3,23.0,-3374,1989,5363,0,2020-07-19T15:00:00Z,1920
6,Che_Adams,37,4,24,Southampton,Bournemouth,0.486089,0.613218,False,7,...,26628,2,0,58.0,-136,653,789,0,2020-07-19T13:00:00Z,1920
7,Mahmoud Ahmed_Ibrahim Hassan,37,3,90,Aston Villa,Arsenal,0.532708,1.496432,True,10,...,87528,0,1,34.0,3301,4651,1350,0,2020-07-21T19:15:00Z,1920
8,Tyreece_John-Jules,37,4,0,Arsenal,Aston Villa,1.496432,0.532708,False,0,...,470,0,1,0.0,25,68,43,0,2020-07-21T19:15:00Z,1920
9,Adrian_Mariappa,37,2,90,Watford,Manchester City,0.504811,2.468858,True,-1,...,18316,4,0,0.0,-269,120,389,1,2020-07-21T17:00:00Z,1920


In [11]:
df_latest.shape

(665, 34)

In [12]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [13]:
last_play_proba.head()

,player,play_proba
0,Ahmed_El Mohamady,1.00
1,James_Chester,0.00
2,Neil_Taylor,0.75
3,Kortney_Hause,0.75
4,Jonathan_Kodjia,0.00


In [14]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [15]:
df_latest.drop_duplicates(inplace=True)

In [16]:
df_train.drop_duplicates(inplace=True)

In [17]:
df_train.shape

(89106, 35)

In [18]:
df_latest.shape

(665, 35)

In [19]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [20]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
89766,Tommy_Doyle,37,3,0,Manchester City,Watford,2.468858,0.504811,False,0,...,4,0,0.0,-1,14,15,0,2020-07-21T17:00:00Z,1920,1.0
89767,Joseph_Anang,37,1,0,West Ham United,Manchester United,0.681394,1.640220,False,0,...,1,1,0.0,-51,97,148,0,2020-07-22T17:00:00Z,1920,1.0
89768,Erik_Pieters,37,2,90,Burnley,Norwich,0.360601,0.311992,False,12,...,2,0,33.0,56140,58736,2596,0,2020-07-18T16:30:00Z,1920,1.0
89769,Japhet_Tanganga,37,2,0,Tottenham Hotspur,Leicester City,1.737985,1.044531,True,0,...,0,3,0.0,10367,14553,4186,0,2020-07-19T15:00:00Z,1920,1.0
89770,Ravel_Morrison,37,3,0,Sheffield United,Everton,0.300107,1.032597,True,0,...,1,0,0.0,-12,5,17,0,2020-07-20T17:00:00Z,1920,0.0


In [21]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v3.csv')

In [22]:
# df_train_new = pd.read_csv(path/'train_v3.csv', index_col=0, dtype={'season':str})

In [23]:
# df_train_new[(df_train_new['team'] == 'Manchester United') & 
#              (df_train_new['gw'] == 32) & 
#              (df_train_new['season'] == '1920')]

In [24]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [25]:
fixtures.head(20)

,home_team,away_team,gw
370,Arsenal,Watford,38
371,Burnley,Brighton and Hove Albion,38
372,Chelsea,Wolverhampton Wanderers,38
373,Crystal Palace,Tottenham Hotspur,38
374,Everton,Bournemouth,38
375,Leicester City,Manchester United,38
376,Manchester City,Norwich,38
377,Newcastle United,Liverpool,38
378,Southampton,Sheffield United,38
379,West Ham United,Aston Villa,38


In [26]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [27]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,38,Arsenal,Watford,3,57
1,38,Burnley,Brighton and Hove Albion,90,36
2,38,Chelsea,Wolverhampton Wanderers,8,39
3,38,Crystal Palace,Tottenham Hotspur,31,6
4,38,Everton,Bournemouth,11,91


In [28]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [29]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [30]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,38,Watford,Arsenal,José_Holebas,2,46,0,False
1,38,Watford,Arsenal,Christian_Kabasele,2,43,100,False
2,38,Watford,Arsenal,Adam_Masina,2,43,100,False
3,38,Watford,Arsenal,Adrian_Mariappa,2,41,100,False
4,38,Watford,Arsenal,Francisco_Femenía Far,2,42,100,False


In [31]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,38,Arsenal,Watford,Shkodran_Mustafi,2,51,0,True
1,38,Arsenal,Watford,Héctor_Bellerín,2,52,100,True
2,38,Arsenal,Watford,Sead_Kolasinac,2,52,100,True
3,38,Arsenal,Watford,Ainsley_Maitland-Niles,2,45,100,True
4,38,Arsenal,Watford,Sokratis_Papastathopoulos,2,48,100,True


In [32]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 29.6,
 'Aaron_Cresswell': 86.8,
 'Aaron_Mooy': 53.6,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 76.0,
 'Abdoulaye_Doucouré': 84.4,
 'Adalberto_Peñaranda': 0.0,
 'Adam_Idah': 14.6,
 'Adam_Lallana': 0.0,
 'Adam_Masina': 50.8,
 'Adam_Smith': 69.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 71.6,
 'Adrian_Mariappa': 45.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 40.8,
 'Ainsley_Maitland-Niles': 9.8,
 'Albian_Ajeti': 0.8,
 'Alex_Iwobi': 35.25,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 40.0,
 'Alexander_Tettey': 83.75,
 'Alexandre Nascimento_Costa Silva': 0.0,
 'Alexandre_Jankewitz': 0.0,
 'Alexandre_Lacazette': 62.8,
 'Alexis_Mac Allister': 25.4,
 'Alfie_Lewis': 0.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 4.0,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 52.666666666666664,
 'Andre_Gray': 4.8,
 'Andreas_Christensen': 45.0,
 'Andreas_Pereira': 0.0,
 'Andrew_Robertson': 81.0,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 18

In [33]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [34]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,38,Arsenal,Watford,Shkodran_Mustafi,2,5.1,0.0,True,1.496432,0.504811,1920,0.000000
1,38,Arsenal,Watford,Héctor_Bellerín,2,5.2,1.0,True,1.496432,0.504811,1920,37.400000
2,38,Arsenal,Watford,Sead_Kolasinac,2,5.2,1.0,True,1.496432,0.504811,1920,71.600000
3,38,Arsenal,Watford,Ainsley_Maitland-Niles,2,4.5,1.0,True,1.496432,0.504811,1920,9.800000
4,38,Arsenal,Watford,Sokratis_Papastathopoulos,2,4.8,1.0,True,1.496432,0.504811,1920,0.000000
5,38,Arsenal,Watford,Nacho_Monreal,2,5.0,0.0,True,1.496432,0.504811,1920,0.000000
6,38,Arsenal,Watford,Laurent_Koscielny,2,5.0,0.0,True,1.496432,0.504811,1920,0.000000
7,38,Arsenal,Watford,Konstantinos_Mavropanos,2,4.3,0.0,True,1.496432,0.504811,1920,0.000000
8,38,Arsenal,Watford,Carl_Jenkinson,2,4.5,0.0,True,1.496432,0.504811,1920,0.000000
9,38,Arsenal,Watford,Rob_Holding,2,4.4,1.0,True,1.496432,0.504811,1920,36.000000


In [35]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')